In [1]:
from o2s.models.object_2_sphere import Mesh2Sphere
from o2s.datasets.radar_dataset import RadarDataset
from o2s.datasets.transforms.general import Compose
from o2s.datasets.transforms.radar import Log, Abs, Normalize, Center
from o2s.datasets.transforms.mesh import MeshNormalize

import numpy as np
import matplotlib.pyplot as plt
from torch_harmonics import plotting
from torch_geometric.loader import DataLoader
import xarray
import torch
from torch import nn

from hydra_zen import load_from_yaml, instantiate, get_target
from mlflow import MlflowClient, set_tracking_uri


In [2]:
cfg = load_from_yaml('/home/colin/hdd/workspace/object2sphere/config/rem_config.yaml')
module_equi = get_target(cfg.module)
module_equi = module_equi.load_from_checkpoint(
    '/home/colin/hdd/workspace/object2sphere/model_epoch=19-v1.ckpt',
    backbone=instantiate(cfg.module.backbone),
    criterion=instantiate(cfg.module.criterion),
    optim=instantiate(cfg.module.optim),
)

/home/colin/anaconda3/envs/torch/lib/python3.10/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/home/colin/anaconda3/envs/torch/lib/python3.10/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/home/colin/anaconda3/envs/torch/lib/python3.10/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/

In [3]:
def convert_log_center_to_db(data, std=cfg.test_dataset.transform.transforms[2].std, mean=cfg.test_dataset.transform.transforms[2].mean):
    data = data*std
    data = data + mean
    data = data *10
    return data
    
test_ds_fp = '/home/colin/hdd/workspace/object2sphere/datasets/test_dataset.nc'
ds = RadarDataset(
    test_ds_fp,
    'test',
    seed=0,
    transform=instantiate(cfg.test_dataset.transform),
    shuffle_before_split=False,
    mesh_mode='simple',
    orientation_mode='full',
)

ConfigIndexError: list index out of range
    full_key: test_dataset.transform.transforms[2]
    object_type=list

In [ ]:
dl = DataLoader(ds, batch_size=2, num_workers=0, shuffle=False)
itr = iter(dl)

In [ ]:
mesh, resp   = next(itr)
#target = convert_log_center_to_db(resp[0,53,:,:])
target = resp[0,53,:,:]
with torch.no_grad():
    pred, w = module_equi(mesh.cuda())
#pred = convert_log_center_to_db(pred)
print(torch.mean((target.cuda() - pred)**2))

In [ ]:
plotting.plot_spherical_fn(pred[0].cpu().squeeze().numpy(), fig=plt.figure(figsize=(2,2)), vmin=target.cpu().min(), vmax=target.cpu().max())

In [ ]:
import matplotlib.animation as animation

fig = plt.figure(layout='constrained', figsize=(12, 8), dpi=72)

moviewriter = animation.writers['pillow'](fps=30)
moviewriter.setup(fig, 'pred.gif', dpi=72)

num_frames = 90
for frame in range(num_frames):
    # compute the rotation of the sphere
    lon = -num_frames - (360 / num_frames) * frame

    if frame % 10 == 0:
        print(f"frame={frame}")
    
        plotting.plot_spherical_fn(pred[0].cpu().squeeze().numpy(), fig=fig, central_longitude=lon, vmin=target.cpu().min(), vmax=target.cpu().max())

    plt.draw()
    moviewriter.grab_frame()

moviewriter.finish()

In [ ]:
plotting.plot_spherical_fn(target.cpu().squeeze().numpy(), fig=plt.figure(figsize=(2,2)), vmin=target.cpu().min(), vmax=target.cpu().max())

In [ ]:
fig = plt.figure(layout='constrained', figsize=(12, 8), dpi=72)

moviewriter = animation.writers['pillow'](fps=30)
moviewriter.setup(fig, 'target.gif', dpi=72)

num_frames = 90
for frame in range(num_frames):
    # compute the rotation of the sphere
    lon = -num_frames - (360 / num_frames) * frame

    if frame % 10 == 0:
        print(f"frame={frame}")
    
        plotting.plot_spherical_fn(target.cpu().squeeze().numpy(), fig=fig, central_longitude=lon, vmin=target.cpu().min(), vmax=target.cpu().max())

    plt.draw()
    moviewriter.grab_frame()

moviewriter.finish()

In [ ]:
plotting.plot_spherical_fn(np.abs(target.cpu().squeeze().numpy() - pred[0].cpu().squeeze().numpy()), fig=plt.figure(figsize=(2,2)))

fig = plt.figure(layout='constrained', figsize=(12, 8), dpi=72)

moviewriter = animation.writers['pillow'](fps=30)
moviewriter.setup(fig, 'error.gif', dpi=72)

num_frames = 360
for frame in range(num_frames):
    # compute the rotation of the sphere
    lon = -num_frames - (360 / num_frames) * frame

    if frame % 10 == 0:
        print(f"frame={frame}")
    
        plotting.plot_spherical_fn(np.abs(target.cpu().squeeze().numpy() - pred.cpu().squeeze().numpy()), fig=fig, central_longitude=lon)

    plt.draw()
    moviewriter.grab_frame()

moviewriter.finish()

## Ground Truth
![SegmentLocal](target.gif "segment")

## Prediction
![SegmentLocal](pred.gif "segment")

## Error
![SegmentLocal](error.gif "segment")

## Ground Truth
![SegmentLocal](target.gif "segment")

## Prediction
![SegmentLocal](pred.gif "segment")

## Error
![SegmentLocal](error.gif "segment")